In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pip install requests beautifulsoup4 pandas openpyxl scholarly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 68.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.6 MB/s eta 0:00:00
  Created wheel for bibtexparser: filename=bibtexparser-1.4.1-py3-none-any.whl size=43252 sha256=dff3785eedb2e68727e0924f39071f768efa921a06ec337ea930942340fa45ed
  Stored in directory: /root/.cache/pip/wheels/08/c6/c3/56e639fab68d1fdbf13ea147636d9795ccdbd3c1d3178d1332
  Created wheel for free-proxy: filename=free_proxy-1.1.1-py3-none-any.whl size=5642 sha256=ecac4330b813a03f5b1ab813061c3407c30c5e4daecb0472a2cd49e526c9b23f
  Stored in directory: /root/.cache/pip/wheels/5a/96/c7/5a434714fff4fea9a59075428b142626e0a74f8c3bf90a50d0
Successfully built bibtexparser free-proxy
  Attempting un

In [2]:
pip install cloudscraper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numba cupy


Note: you may need to restart the kernel to use updated packages.


In [6]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import concurrent.futures
from numba import cuda

# Base URL of the Stanford University page on AD Scientific Index
base_url = "https://www.adscientificindex.com"

# List to store all scientists' data
data = []

# Function to parse the scientist data from a single page
def parse_page(soup):
    table = soup.find('table', {'class': 'table table-striped table-bordered table-sm'})
    if table:
        rows = table.find_all('tr')
        for row in rows[1:]:
            cols = row.find_all('td')
            parsed_cols = []
            profile_url = ""
            for col in cols:
                if col.find('a'):
                    a_tag = col.find('a')
                    if 'subject=' in a_tag['href']:
                        subject = col.find('a').text.strip()
                        parsed_cols.append(subject)
                    else:
                        for sup in a_tag.find_all('sup'):
                            sup.decompose()
                        name = a_tag.get_text(strip=True)
                        parsed_cols.append(name)
                        if "/scientist/" in a_tag['href']:
                            profile_url = base_url + a_tag['href']
                else:
                    text_parts = col.get_text(strip=True).split()
                    parsed_cols.append(text_parts[0] if text_parts else '')
            parsed_cols.append(profile_url)
            if parsed_cols not in data:
                data.append(parsed_cols)
    else:
        print("Table not found on this page")

# Function to fetch Google Scholar URL from a professor's profile page
def fetch_google_scholar_url(profile_url):
    if not profile_url:
        return ''
    response = scraper.get(profile_url)
    profile_soup = BeautifulSoup(response.content, 'html.parser')
    google_scholar_div = profile_soup.find('div', class_='id')
    if google_scholar_div:
        a_tag = google_scholar_div.find('a', href=True, class_='google_scholar')
        if a_tag:
            return a_tag['href']
    return ''

# Create a cloudscraper instance
scraper = cloudscraper.create_scraper()

# Function to scrape a single page and parse it
def scrape_page(page):
    url = f"https://www.adscientificindex.com/?s={page}&country_code=in"
    response = scraper.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    parse_page(soup)

# Loop through each page to scrape the data concurrently
page_ranges = range(0, 20001, 50)  # Adjusted range for example, use full range as needed
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(scrape_page, page_ranges)

# Fetch Google Scholar URLs for each professor concurrently
profile_urls = [row[-1] for row in data]
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    google_scholar_urls = list(executor.map(fetch_google_scholar_url, profile_urls))

# Add Google Scholar URLs to data
for i in range(len(data)):
    data[i].append(google_scholar_urls[i])

# Ensure all rows have the same number of columns
max_columns = max(len(row) for row in data)
for row in data:
    while len(row) < max_columns:
        row.append('')

# Define column names dynamically based on the maximum number of columns found
columns = ["Column_" + str(i) for i in range(1, max_columns - 1)]
columns.append("Profile_URL")
columns.append("Google_Scholar_URL")

# Create a DataFrame and save it to a CSV file
df = pd.DataFrame(data, columns=columns)
df.to_csv('stanford_scientists_with_google_scholar_urls_0to20000.csv', index=False)

print("Data has been scraped and saved to 'stanford_scientists_with_google_scholar_urls.csv'")

# Simulated heavy computation using GPU (e.g., data transformation) after scraping
@cuda.jit
def heavy_computation_gpu(arr, result):
    idx = cuda.grid(1)
    if idx < arr.size:
        result[idx] = arr[idx] ** 2

# Convert data to a NumPy array for demonstration
data_array = np.random.rand(len(data), max(len(row) for row in data)).flatten()

# Allocate memory on the GPU
d_data_array = cuda.to_device(data_array)
d_result = cuda.device_array_like(d_data_array)

# Define number of threads and blocks
threads_per_block = 256
blocks_per_grid = (d_data_array.size + (threads_per_block - 1)) // threads_per_block

# Perform heavy computation using GPU
heavy_computation_gpu[blocks_per_grid, threads_per_block](d_data_array, d_result)

# Copy result back to host
result = d_result.copy_to_host()

# Sum the results (if needed)
final_result = np.sum(result)
print(f"Result of heavy computation: {final_result}")


Data has been scraped and saved to 'stanford_scientists_with_google_scholar_urls.csv'
Result of heavy computation: 91419.93258792865


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive"
}

# Function to get profile details
def get_profile_details(soup):
    profile_name = soup.find("div", id="gsc_prf_in").text.strip()
    return profile_name

# Function to get the research papers and journals
def get_research_papers(soup):
    papers = []
    paper_table = soup.find_all("tr", class_="gsc_a_tr")
    for paper in paper_table:
        title_elem = paper.find("a", class_="gsc_a_at")
        authors_journal_elem = paper.find_all("div", class_="gs_gray")
        if len(authors_journal_elem) >= 2:
            publication = authors_journal_elem[1].text.strip()
        else:
            publication = ""
        if title_elem:
            title = title_elem.text.strip()
            papers.append((title, publication))
    return papers

# Function to load all research papers by handling pagination
def load_all_papers(url, headers):
    research_papers = []
    start = 0
    while True:
        try:
            response = requests.get(url + f"&cstart={start}&pagesize=100", headers=headers)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            break
        soup = BeautifulSoup(response.content, "html.parser")
        papers = get_research_papers(soup)
        if not papers:
            break
        research_papers.extend(papers)
        start += len(papers)
        time.sleep(20)
    return research_papers

# Function to process each URL
def process_url(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        profile_name = get_profile_details(soup)
        research_papers = load_all_papers(url, headers)
        data = []
        for paper in research_papers:
            data.append({
                "Profile Name": profile_name,
                "Research Paper": paper[0],
                "Journal": paper[1]
            })
        return data
    except requests.exceptions.RequestException as e:
        print(f"Request error for {url}: {e}")
        return []

# Load the test CSV
file_path = '/kaggle/input/test-10001-10500/test.csv'
test_df = pd.read_csv(file_path)

# Prepare to collect all data
all_data = []

# Use ThreadPoolExecutor to fetch data concurrently
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(process_url, row['scholar_link']): row['scholar_link'] for index, row in test_df.iterrows()}
    for future in as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
            all_data.extend(data)
        except Exception as e:
            print(f"Error processing {url}: {e}")

# Create a DataFrame from the collected data
result_df = pd.DataFrame(all_data)

# Save the results to a new CSV file
result_df.to_csv('professors_google_scholar_profiles_upto10500.csv', index=False)

print("Data collection complete. Results saved to 'professors_google_scholar_profiles_fortest.csv'.")


Request error for https://scholar.google.co.th/citations?hl=en&user=12vqBQcAAAAJ: 404 Client Error: Not Found for url: https://scholar.google.co.th/citations?hl=en&user=12vqBQcAAAAJ
Error processing https://scholar.google.co.th/citations?hl=en&user=https://scholar.google.com/citations?user=dDZIkMQAAAAJ%26hl=tr: 'NoneType' object has no attribute 'text'
Request error for https://scholar.google.co.th/citations?hl=en&user=OKjj9XMAAAAJ: 404 Client Error: Not Found for url: https://scholar.google.co.th/citations?hl=en&user=OKjj9XMAAAAJ
Request error for https://scholar.google.co.th/citations?hl=en&user=BIPIe2AAAAAJ: 404 Client Error: Not Found for url: https://scholar.google.co.th/citations?hl=en&user=BIPIe2AAAAAJ
Request error for https://scholar.google.co.th/citations?hl=en&user=Ax3JVNkAAAAJ: 404 Client Error: Not Found for url: https://scholar.google.co.th/citations?hl=en&user=Ax3JVNkAAAAJ
Error processing https://scholar.google.co.th/citations?hl=en&user=https://scholar.google.com/citat